In [37]:
import cv2
import numpy as np
import pkg_resources
import tensorflow as tf
from tensorflow.python.saved_model import tag_constants
import os

FACES_PATH = '../data/face_detection/faces/'


In [55]:
class PNet(tf.keras.Model):
    def __init__(self):
        super(PNet, self).__init__(name="PNet")
        # Define layers here.
        self.conv1 = tf.keras.layers.Conv2D(10, (3, 3), name="conv1")
        self.prelu1 = tf.keras.layers.PReLU(tf.constant_initializer(0.25), shared_axes=[1, 2], name="prelu1")
        self.pool1 = tf.keras.layers.MaxPooling2D((2, 2), name="pool1")
        self.conv2 = tf.keras.layers.Conv2D(16, (3, 3), name="conv2")
        self.prelu2 = tf.keras.layers.PReLU(tf.constant_initializer(0.25), shared_axes=[1, 2], name="prelu2")
        self.conv3 = tf.keras.layers.Conv2D(32, (3, 3), name="conv3")
        self.prelu3 = tf.keras.layers.PReLU(tf.constant_initializer(0.25), shared_axes=[1, 2], name="prelu3")
        self.cls_output = tf.keras.layers.Conv2D(2, (1, 1), activation="softmax", name="conv4-1")
        self.bbox_pred = tf.keras.layers.Conv2D(4, (1, 1), name="conv4-2")
        #self.landmark_pred = keras.layers.Conv2D(10, (1, 1), name="conv4_3")

    def call(self, inputs):
        # Define your forward pass here,
        # using layers you previously defined (in `__init__`).
        scores = None

        x = self.conv1(inputs)
        x = self.prelu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.prelu2(x)
        x = self.conv3(x)
        x = self.prelu3(x)
        scores = [self.cls_output(x), self.bbox_pred(x)]#, self.landmark_pred(x)]
        
        return scores


## Dataset iterator

In [56]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y
        
        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))


In [57]:
# Set up some global variables
USE_GPU = False

if USE_GPU:
    device = '/device:GPU:0'
else:
    device = '/cpu:0'

# Constant to control how often we print when training models
print_every = 100

print('Using device: ', device)

Using device:  /cpu:0


Test the PNet  to ensure that the implementation does not crash and produces outputs of the expected shape.
Pnet will output are:
1. Face classification,  size (batch,1,1,2) for 2 calss classification, "Face", and "Not face"
2. Bounding box  (batch,1,1,4) for 4 boundind box corrdinates (x,y,w,h)

In [58]:
def test_PNet(batch=64):    
    model = PNet()
    with tf.device(device):
        x = tf.zeros((batch, 12, 12, 3))
        classification_scores, bbox_score = model(x)
        print(model.summary())
        print('\nP-Net output size testing: \nclassificatin score output', classification_scores.shape,
              '\nbounsing box score output', bbox_score.shape)

batch_test = 32
test_PNet(batch_test)

Model: "PNet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               multiple                  280       
_________________________________________________________________
prelu1 (PReLU)               multiple                  10        
_________________________________________________________________
pool1 (MaxPooling2D)         multiple                  0         
_________________________________________________________________
conv2 (Conv2D)               multiple                  1456      
_________________________________________________________________
prelu2 (PReLU)               multiple                  16        
_________________________________________________________________
conv3 (Conv2D)               multiple                  4640      
_________________________________________________________________
prelu3 (PReLU)               multiple                  32     

In [59]:
def read_pos_images():
    #Read positive images:
    path, __, filenames = next(os.walk(FACES_PATH+'pos_train/'))
    file_count = 2048 #len(filenames)
    images = np.empty([0,12,3])
    for i in range(file_count):
        j=i+1
        img=cv2.imread(f"{path}{j}.bmp")
        images=np.append(images,img,axis=0)
    #Create list of probabilities:
    prob=[]
    for i in range(file_count):
        prob.append([[[0.0,1.0]]])
    #Create list of coordinates:
    coordinates=[]
    file = open(FACES_PATH+'coordinates.txt','r')
    lines = file.readlines()
    lines = [line[:-1] for line in lines]
    idx=[1,0,3,2]
    for line in lines:
        line = line.split(" ")
        line = line[1]
        line=line[1:-1]
        line = line.split(",")
        #Transpose coordinates
        x=0
        nline=[]
        for i in idx:
            nline.append(line[i])
            x=x+1
        line=[[[float(c) for c in nline]]]
        coordinates.append(line)
    #Return images, probs, and coordinates
    return images, prob, coordinates

def read_neg_images():
    #Read negative images:
    path, __, filenames = next(os.walk(FACES_PATH+'neg_train/'))
    file_count = 2048 #len(filenames)
    images = np.empty([0,12,3])
    for i in range(file_count):
        j=i+1
        img=cv2.imread(f"{path}{j}.bmp")
        images=np.append(images,img,axis=0)
    #Create list of probabilities:
    prob=[]
    for i in range(file_count):
        prob.append([[[1.0,0.0]]])
    #Create list of coordinates:
    coordinates=[]
    for i in range(file_count):
        coordinates.append([[[0.0,0.0,0.0,0.0]]])
    #Return images, prob, coordinates
    return images, prob, coordinates

#Read in all images, probabilities, and coordinates
pimages, pprob, pcoordinates = read_pos_images()
nimages, nprob, ncoordinates = read_neg_images()
o_images=np.append(pimages,nimages,axis=0)
o_images=np.reshape(o_images,(-1,12,12,3))
o_prob=pprob+nprob
o_coordinates=pcoordinates+ncoordinates

#Shuffle them up using an index
idx=np.arange(len(o_prob))
np.random.shuffle(idx)
images=np.empty_like(o_images)
c=0
for i in idx:
    images[c]=o_images[i]
    c=c+1
images=(np.float32)(images-127.5)/128.0
images = np.transpose(images, (0, 2, 1, 3)) #Transpose images
prob=[]
for i in idx:
    prob.append(o_prob[i])
coordinates=[]
for i in idx:
    coordinates.append(o_coordinates[i])

In [60]:
print('X_train , Image batch shape ', images.shape)
print('y_train , Classification ground true batch shape ' ,np.array(prob).shape)
print('y_train , Coordinates ground true batch shape ', np.array(coordinates).shape)

X_train , Image batch shape  (4096, 12, 12, 3)
y_train , Classification ground true batch shape  (4096, 1, 1, 2)
y_train , Coordinates ground true batch shape  (4096, 1, 1, 4)


## Create X_data for train and validation

In [61]:
X_data = images

In [62]:
print('X_data shape',X_data.shape)

X_data shape (4096, 12, 12, 3)


In [63]:
X_data.min()

-0.99609375

## Create "y_data" for train and validation

In [64]:
y_data = np.concatenate((np.array(prob), np.array(coordinates)), axis=3)

In [65]:
print('y_data shape',y_data.shape)

y_data shape (4096, 1, 1, 6)


In [66]:
print('y_data Classification shape', y_data[:,:,:,:2].shape)
print('y_data Coordinate shape',y_data[:,:,:,2:].shape)

y_data Classification shape (4096, 1, 1, 2)
y_data Coordinate shape (4096, 1, 1, 4)


## Divide dataset to "train', "val" and "test"

In [67]:
def load_data(X, y, training_prec = 0.7, val_prec = 0.1, test_prec = 0.2):
        data_length = len(X)
        num_training = np.int(data_length * training_prec)
        num_validation = np.int(data_length * val_prec)
        
        mask = range(num_training)
        X_train = X[mask]
        y_train = y[mask]
        mask = range(num_training, num_training + num_validation)
        X_val = X[mask]
        y_val = y[mask]
        mask = range(num_training + num_validation, data_length)
        X_test = X[mask]
        y_test = y[mask]
        
        return X_train, y_train, X_val, y_val, X_test, y_test


In [68]:
X_train, y_train, X_val, y_val, X_test, y_test = load_data(X_data, y_data)
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (2867, 12, 12, 3)
Train labels shape:  (2867, 1, 1, 6) float64
Validation data shape:  (409, 12, 12, 3)
Validation labels shape:  (409, 1, 1, 6)
Test data shape:  (820, 12, 12, 3)
Test labels shape:  (820, 1, 1, 6)


In [69]:
train_dset = Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=64, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=64)

## Lets test a single batch 

In [ ]:
def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.Adam(learning_rate) 
    #return tf.keras.optimizers.SGD(lr=learning_rate, momentum=0.9, nesterov=True)

In [ ]:
model = PNet()
optimizer = optimizer_init_fn()

In [ ]:
classification_loss = tf.keras.losses.BinaryCrossentropy()
bbox_loss = tf.keras.losses.MeanSquaredError()

In [ ]:
train_loss = tf.keras.metrics.BinaryCrossentropy(name='train_classification_loss')
train_bbox_loss = tf.keras.metrics.MeanSquaredError(name='train_bbox_loss')


In [ ]:
train_accuracy = tf.keras.metrics.BinaryAccuracy(name='train_accuracy')

### Forward pass loss

In [ ]:
with tf.GradientTape() as tape:
    classification_scores, bbox_scores = model(X_train , training=True)
    prediction_loss = classification_loss(y_data[:,:,:,:2], classification_scores)
    coordinate_loss = bbox_loss(y_data[:,:,:,2:], bbox_scores)
    loss = prediction_loss + 0.5 * coordinate_loss
    # Print loss 
    print('Classification loss',prediction_loss)
    print(coordinate_loss)
    print(loss)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    # Update the metrics
    train_loss.update_state(y_data[:,:,:,:2], classification_scores)
    train_bbox_loss.update_state(y_data[:,:,:,2:], bbox_scores)
    train_accuracy.update_state(y_data[:,:,:,:2], classification_scores)

In [ ]:
template = 'Loss: {}, Accuracy: {}'
print (template.format(train_loss.result(),
                       train_accuracy.result()*100))

In [73]:
def train(model_init_fn, optimizer_init_fn, num_epochs=1, is_training=False):
    """
    Simple training loop for use with models defined using tf.keras. It trains
    a model for one epoch on  training set and periodically checks
    accuracy on the validation set.
    
    Inputs:
    - model_init_fn: A function that takes no parameters; when called it
      constructs the model we want to train: model = model_init_fn()
    - optimizer_init_fn: A function which takes no parameters; when called it
      constructs the Optimizer object we will use to optimize the model:
      optimizer = optimizer_init_fn()
    - num_epochs: The number of epochs to train for
    
    Returns: Nothing, but prints progress during trainingn
    """    
    with tf.device(device):
        #compute the loss function over the classification and ovr bounding box 
        classification_loss = tf.keras.losses.BinaryCrossentropy()
        bbox_loss = tf.keras.losses.MeanSquaredError()        
        
        model = PNet()
        optimizer = optimizer_init_fn()
        
        train_loss = tf.keras.metrics.BinaryCrossentropy(name='train_classification_loss')
        train_bbox_loss = tf.keras.metrics.MeanSquaredError(name='train_bbox_loss')
        
        train_accuracy = tf.keras.metrics.BinaryAccuracy(name='train_accuracy')
            
        #val_loss = tf.keras.metrics.Mean(name='val_loss')
        val_loss = tf.keras.metrics.BinaryCrossentropy(name='val_classification_loss')
        val_bbox_loss = tf.keras.metrics.MeanSquaredError(name='val_bbox_loss')

        val_accuracy = tf.keras.metrics.BinaryAccuracy(name='val_accuracy')
        
        t = 0
        for epoch in range(num_epochs):
            
            # Reset the metrics - https://www.tensorflow.org/alpha/guide/migration_guide#new-style_metrics
            train_loss.reset_states()
            train_bbox_loss.reset_states()
            
            train_accuracy.reset_states()
            
            for x_np, y_np in train_dset:
                with tf.GradientTape() as tape:
                    
                    # Use the model function to build the forward pass.
                    classification_scores, bbox_scores = model(x_np , training=True)
                    prediction_loss = classification_loss(y_np[:,:,:,:2], classification_scores)
                    coordinate_loss = bbox_loss(y_np[:,:,:,2:], bbox_scores)
                    loss = prediction_loss + 0.5 * coordinate_loss
                    # Print loss 
                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
                    # Update the metrics
                    train_loss.update_state(y_np[:,:,:,:2], classification_scores)
                    train_bbox_loss.update_state(y_np[:,:,:,2:], bbox_scores)
                    train_accuracy.update_state(y_np[:,:,:,:2], classification_scores)

                   
                    if t % print_every == 0:
                        val_loss.reset_states()
                        val_accuracy.reset_states()
                        for test_x, test_y in val_dset:
                            # During validation at end of epoch, training set to False
                            classification_scores, bbox_scores = model(test_x, training=False)
                            t_prediction_loss = classification_loss(test_y[:,:,:,:2], classification_scores)
                            t_coordinate_loss = bbox_loss(test_y[:,:,:,2:], bbox_scores)
                            t_loss = t_prediction_loss + 0.5 * t_coordinate_loss

                            val_loss.update_state(test_y[:,:,:,:2], classification_scores)
                            val_bbox_loss.update_state(test_y[:,:,:,2:], bbox_scores)
                            val_accuracy.update_state(test_y[:,:,:,:2], classification_scores)
                        
                        template = 'Iteration {}, Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
                        print (template.format(t, epoch+1,
                                             train_loss.result(),
                                             train_accuracy.result()*100,
                                             val_loss.result(),
                                             val_accuracy.result()*100))
                    t += 1

In [74]:
#device = '/device:GPU:0'   # Change this to a CPU/GPU as you wish!
device = '/cpu:0'        # Change this to a CPU/GPU as you wish!
print_every = 1
num_epochs = 3

model = PNet()

def model_init_fn():
    return PNet()

def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.Adam(learning_rate) 
    #return tf.keras.optimizers.SGD(lr=learning_rate, momentum=0.9, nesterov=True)

train(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

Iteration 0, Epoch 1, Loss: 0.6764088869094849, Accuracy: 56.25, Val Loss: 0.6777464151382446, Val Accuracy: 57.94621276855469
Iteration 1, Epoch 1, Loss: 0.668533205986023, Accuracy: 60.15625, Val Loss: 0.6744998693466187, Val Accuracy: 56.234718322753906
Iteration 2, Epoch 1, Loss: 0.6670225262641907, Accuracy: 59.89583206176758, Val Loss: 0.6720108985900879, Val Accuracy: 55.99021911621094
Iteration 3, Epoch 1, Loss: 0.6783003807067871, Accuracy: 56.640625, Val Loss: 0.6694439053535461, Val Accuracy: 55.74571990966797
Iteration 4, Epoch 1, Loss: 0.677046000957489, Accuracy: 56.25, Val Loss: 0.6670352220535278, Val Accuracy: 55.99021911621094
Iteration 5, Epoch 1, Loss: 0.676058828830719, Accuracy: 55.72916793823242, Val Loss: 0.6645098924636841, Val Accuracy: 55.99021911621094
Iteration 6, Epoch 1, Loss: 0.6784628033638, Accuracy: 55.13392639160156, Val Loss: 0.6619030833244324, Val Accuracy: 55.99021911621094
Iteration 7, Epoch 1, Loss: 0.6826684474945068, Accuracy: 54.1015625, Val

Iteration 61, Epoch 2, Loss: 0.4952158033847809, Accuracy: 75.73529815673828, Val Loss: 0.42307302355766296, Val Accuracy: 81.90708923339844
Iteration 62, Epoch 2, Loss: 0.4899347126483917, Accuracy: 75.78125, Val Loss: 0.4222240447998047, Val Accuracy: 82.1515884399414
Iteration 63, Epoch 2, Loss: 0.48909100890159607, Accuracy: 75.74012756347656, Val Loss: 0.41322988271713257, Val Accuracy: 82.88508605957031
Iteration 64, Epoch 2, Loss: 0.4824223518371582, Accuracy: 76.171875, Val Loss: 0.40260744094848633, Val Accuracy: 82.1515884399414
Iteration 65, Epoch 2, Loss: 0.4736040532588959, Accuracy: 76.93452453613281, Val Loss: 0.39473825693130493, Val Accuracy: 83.61858367919922
Iteration 66, Epoch 2, Loss: 0.47343331575393677, Accuracy: 76.84659576416016, Val Loss: 0.3890717327594757, Val Accuracy: 84.3520736694336
Iteration 67, Epoch 2, Loss: 0.4727829694747925, Accuracy: 76.97010803222656, Val Loss: 0.3883194625377655, Val Accuracy: 83.12958526611328
Iteration 68, Epoch 2, Loss: 0.466

Iteration 121, Epoch 3, Loss: 0.39132511615753174, Accuracy: 81.34765625, Val Loss: 0.3305686414241791, Val Accuracy: 85.330078125
Iteration 122, Epoch 3, Loss: 0.3868938684463501, Accuracy: 81.7234878540039, Val Loss: 0.33431872725486755, Val Accuracy: 84.84107971191406
Iteration 123, Epoch 3, Loss: 0.3911616802215576, Accuracy: 81.43382263183594, Val Loss: 0.33027932047843933, Val Accuracy: 85.08557891845703
Iteration 124, Epoch 3, Loss: 0.38915443420410156, Accuracy: 81.5625, Val Loss: 0.32594749331474304, Val Accuracy: 85.57456970214844
Iteration 125, Epoch 3, Loss: 0.3878253698348999, Accuracy: 81.77082824707031, Val Loss: 0.32168149948120117, Val Accuracy: 86.55256652832031
Iteration 126, Epoch 3, Loss: 0.38677746057510376, Accuracy: 81.75675964355469, Val Loss: 0.32063230872154236, Val Accuracy: 86.55256652832031
Iteration 127, Epoch 3, Loss: 0.38380110263824463, Accuracy: 81.90789031982422, Val Loss: 0.32283371686935425, Val Accuracy: 86.79706573486328
Iteration 128, Epoch 3, L

In [103]:
X_test[10:11,:,:,:].shape

(1, 12, 12, 3)

In [107]:
classification_scores, bbox_scores = model(X_train[11:12,:,:,:], training=False)

In [108]:
print(classification_scores)
print(bbox_scores)

tf.Tensor([[[[0.45885938 0.5411406 ]]]], shape=(1, 1, 1, 2), dtype=float32)
tf.Tensor([[[[-0.22488847 -0.08204336  0.14587773 -0.12911837]]]], shape=(1, 1, 1, 4), dtype=float32)


In [106]:
print(y_test[11:12,:,:,:])

[[[[1.         0.         0.25       0.08333333 1.         0.41666667]]]]


In [ ]:
with tf.compat.v1.Session() as sess:
    with tf.Graph().as_default():
        
        #Initialize training
        sess = tf.compat.v1.Session()
        #train_net=PNet(sess)
        train_net=PNet()
        bimg=tf.compat.v1.placeholder(tf.float32, shape=(100,12,12,3))
        bprob=tf.compat.v1.placeholder(tf.float32, shape=(100,1,1,2))
        bprobmask=tf.compat.v1.placeholder(tf.float32, shape=(100,1))
        bcoord=tf.compat.v1.placeholder(tf.float32, shape=(100,1,1,4))
        loss=tf.reduce_mean(tf.square(bprob-train_net.get_layer('conv4-1')))+bprobmask*0.5*tf.reduce_mean(tf.square(bcoord-train_net.get_layer('conv4-2')))
        optimizer = tf.train.AdamOptimizer()
        train = optimizer.minimize(loss)
        init = tf.global_variables_initializer()
        sess.run(init)
        saver = tf.train.Saver()
        
        #Test
        img=cv2.imread("1.bmp")
        img1=(img-127.5)/128.0
        img2=np.expand_dims(img1, 0)
        print(train_net.feed(img2))
        
        #Grab a batch of images, probs, and coordinates, and feed into training
        for j in range(10):
            i=0
            f=100
            while f<len(prob):
                batchimg=images[i:f]
                batchprob=prob[i:f]
                k=np.array(prob[i:f])
                k1=np.reshape(k,(100,2))
                k2=k1[:,1]*1.0
                k3=np.reshape(k2,(100,1))
                batchprobmask=k3
                batchcoord=coordinates[i:f]
                i=i+100
                f=f+100
                sess.run(train,feed_dict={'pnet/input:0': batchimg, bprob: batchprob, bcoord: batchcoord,bprobmask:batchprobmask})
            print(train_net.feed(img2))

        tf.trainable_variables()
        wt=np.load('./data/mtcnn_weights.npy')

        var1=[v for v in tf.trainable_variables() if v.name == "pnet/conv1/weights:0"]
        w1=sess.run(var1)
        wt.all()['PNet']['conv1']['weights']=w1[0]
        var1=[v for v in tf.trainable_variables() if v.name == "pnet/conv1/biases:0"]
        w1=sess.run(var1)
        wt.all()['PNet']['conv1']['biases']=w1[0]
        var1=[v for v in tf.trainable_variables() if v.name == "pnet/prelu1/alpha:0"]
        w1=sess.run(var1)
        wt.all()['PNet']['prelu1']['alpha']=w1[0]
        var1=[v for v in tf.trainable_variables() if v.name == "pnet/prelu2/alpha:0"]
        w1=sess.run(var1)
        wt.all()['PNet']['prelu2']['alpha']=w1[0]
        var1=[v for v in tf.trainable_variables() if v.name == "pnet/prelu3/alpha:0"]
        w1=sess.run(var1)
        wt.all()['PNet']['prelu3']['alpha']=w1[0]
        var1=[v for v in tf.trainable_variables() if v.name == "pnet/conv2/weights:0"]
        w1=sess.run(var1)
        wt.all()['PNet']['conv2']['weights']=w1[0]
        var1=[v for v in tf.trainable_variables() if v.name == "pnet/conv2/biases:0"]
        w1=sess.run(var1)
        wt.all()['PNet']['conv2']['biases']=w1[0]
        var1=[v for v in tf.trainable_variables() if v.name == "pnet/conv3/weights:0"]
        w1=sess.run(var1)
        wt.all()['PNet']['conv3']['weights']=w1[0]
        var1=[v for v in tf.trainable_variables() if v.name == "pnet/conv3/biases:0"]
        w1=sess.run(var1)
        wt.all()['PNet']['conv3']['biases']=w1[0]
        var1=[v for v in tf.trainable_variables() if v.name == "pnet/conv4-1/weights:0"]
        w1=sess.run(var1)
        wt.all()['PNet']['conv4-1']['weights']=w1[0]
        var1=[v for v in tf.trainable_variables() if v.name == "pnet/conv4-1/biases:0"]
        w1=sess.run(var1)
        wt.all()['PNet']['conv4-1']['biases']=w1[0]
        var1=[v for v in tf.trainable_variables() if v.name == "pnet/conv4-2/weights:0"]
        w1=sess.run(var1)
        wt.all()['PNet']['conv4-2']['weights']=w1[0]
        var1=[v for v in tf.trainable_variables() if v.name == "pnet/conv4-2/biases:0"]
        w1=sess.run(var1)
        wt.all()['PNet']['conv4-2']['biases']=w1[0]
        np.save('./data/mtcnn_weights.npy',wt)

In [ ]:
#Testing an image
img=cv2.imread("1.bmp")
img1=(img-127.5)/128.0
img2=np.expand_dims(img1, 0)
print(train_net.feed(img2))

Correct coordinates: 1.bmp: [0.25,0,1,1]